In [1]:
# https://keras.io/examples/mnist_cnn/

In [36]:
import keras
from keras.datasets import mnist
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Flatten,Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

In [37]:
# 参数设置
batch_size =128
num_classes=10
epochs=12
img_rows,img_cols=28,28

In [38]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()

In [39]:
if K.image_data_format() == 'channels_first':
    print(K.image_data_format())
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

In [40]:
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')
x_train/=255.0
x_test/=255.0
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [41]:
y_train

array([5, 0, 4, ..., 5, 6, 8], dtype=uint8)

In [42]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [43]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.]], dtype=float32)

## 创建模型

In [44]:
def create_model1():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=input_shape))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.summary()
    return model
model=create_model1()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               1179776   
_________________________________________________________________
dropout_10 (Dropout)         (None, 128)              

In [45]:
def create_model2():
    img_input=Input(shape=input_shape)
    conv1=Conv2D(32, kernel_size=(3, 3),activation='relu',)(img_input)
    conv2=Conv2D(64, kernel_size=(3, 3), activation='relu')(conv1)
    pool=MaxPooling2D(pool_size=(2, 2))(conv2)
    model=Dropout(0.25)(pool)
    model=Flatten()(model)
    model=Dense(128, activation='relu')(model)
    model=Dropout(0.5)(model)
    preds=Dense(num_classes, activation='softmax')(model)
    model = Model(img_input, preds)
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adadelta(),
                  metrics=['accuracy'])

    model.summary()
    return model
model=create_model2()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 28, 28, 1)         0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
dropout_11 (Dropout)         (None, 12, 12, 64)        0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 9216)              0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               1179

In [ ]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 26s 440us/step - loss: 0.2573 - accuracy: 0.9198 - val_loss: 0.0670 - val_accuracy: 0.9793
Epoch 2/12
60000/60000 [==============================] - 26s 435us/step - loss: 0.0881 - accuracy: 0.9741 - val_loss: 0.0418 - val_accuracy: 0.9862
Epoch 3/12
60000/60000 [==============================] - 27s 444us/step - loss: 0.0695 - accuracy: 0.9790 - val_loss: 0.0412 - val_accuracy: 0.9871
Epoch 4/12
60000/60000 [==============================] - 26s 439us/step - loss: 0.0557 - accuracy: 0.9832 - val_loss: 0.0320 - val_accuracy: 0.9889
Epoch 5/12
27008/60000 [============>.................] - ETA: 13s - loss: 0.0466 - accuracy: 0.9861